In [1]:
from qonnx.util.cleanup import cleanup
from finn.util.visualization import showSrc, showInNetron

# View Original Model in Netron

In [2]:
model_ori = './BED__no_comp__best_mean_f1__cpu.onnx'

In [3]:
showInNetron(model_ori)

Serving './BED__no_comp__best_mean_f1__cpu.onnx' at http://0.0.0.0:8083


# Clean Model with QONNX

In [4]:
model_cleaned = './BED__no_comp__best_mean_f1__cpu__cleaned.onnx'

In [5]:
cleanup(model_ori, out_file=model_cleaned)

In [6]:
showInNetron(model_cleaned)

Stopping http://0.0.0.0:8083
Serving './BED__no_comp__best_mean_f1__cpu__cleaned.onnx' at http://0.0.0.0:8083


# Import Cleaned Model to FINN

In [7]:
from qonnx.core.modelwrapper import ModelWrapper
import qonnx.core.onnx_exec as oxe

In [8]:
model_cleaned_onnx = ModelWrapper(model_cleaned)

# Forward Pass and compare Original and Cleaned Output is the same

In [9]:
import onnxruntime
import numpy as np

In [10]:
dummy_in = np.random.randint(low = 0, high = 255+1, size = (1, 3, 224, 224)) 
dummy_in = (dummy_in / 256.).astype(np.float32)
#dummy_in

In [11]:
#import matplotlib.pyplot as plt

In [12]:
#plt.imshow(dummy_in[0].transpose(1, 2, 0))

## Original Model

In [13]:
ori_session = onnxruntime.InferenceSession(model_ori, providers=["CPUExecutionProvider"])

In [14]:
ori_inputs = {ori_session.get_inputs()[0].name: dummy_in}

In [15]:
ori_yhat = ori_session.run(None, ori_inputs)

In [16]:
ori_yhat

[array([[-4.953125, -4.46875 ]], dtype=float32)]

## Cleaned Model

In [17]:
input_dict = {"global_in": dummy_in}

In [18]:
output_dict = oxe.execute_onnx(model_cleaned_onnx, input_dict)

In [19]:
produced_qonnx = output_dict[list(output_dict.keys())[0]]

In [20]:
produced_qonnx

array([[-4.953125, -4.46875 ]], dtype=float32)

## Comparison

In [21]:
np.isclose(ori_yhat, produced_qonnx).all()

True

# Convert QONNX to FINN-ONNX

In [22]:
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN

In [23]:
model_finn_onnx = ModelWrapper(model_cleaned)

model_finn_onnx = model_finn_onnx.transform(ConvertQONNXtoFINN())

/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/extract_conv_bias.py:52: UserWarning: Could not extract bias from node input: "DequantizeLinear_28_out0"
input: "DequantizeLinear_14_out0"
input: "DequantizeLinear_0_out0"
output: "Conv_0_out0"
name: "Conv_0"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "group"
  i: 1
  type: INT
}
attribute {
  name: "kernel_shape"
  ints: 3
  ints: 3
  type: INTS
}
attribute {
  name: "pads"
  ints: 1
  ints: 1
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}

  warnings.warn(f"Could not extract bias from node {n}")
/home/gmoreno/uav/finn/deps/qonnx/src/qonnx/transformation/extract_conv_bias.py:52: UserWarning: Could not extract bias from node input: "MaxPool_0_out0"
input: "DequantizeLinear_15_out0"
input: "DequantizeLinear_1_out0"
output: "Conv_1_out0"
name: "Conv_1"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1

In [24]:
model_finn = './BED__no_comp__best_mean_f1__cpu__FINN.onnx'

In [25]:
model_finn_onnx.save(model_finn)

In [27]:
showInNetron(model_finn)

Stopping http://0.0.0.0:8083
Serving './BED__no_comp__best_mean_f1__cpu__FINN.onnx' at http://0.0.0.0:8083


In [28]:
model_finn_wrapper = ModelWrapper(model_finn)
input_dict = {"global_in": dummy_in}
output_dict = oxe.execute_onnx(model_finn_wrapper, input_dict)
produced_finn = output_dict[list(output_dict.keys())[0]]

produced_finn

array([[-4.953125, -4.46875 ]], dtype=float32)

In [29]:
np.isclose(produced_qonnx, produced_finn).all()

True